In [21]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib thefuzz pyttsx3 pickle

You should consider upgrading via the 'C:\Users\Joseph\PycharmProjects\FinalYearProject\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [16]:
import os
import cv2
import time
import pickle
import pyttsx3
import numpy as np
import mediapipe as mp
from thefuzz import fuzz
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

In [17]:
mp_holistic = mp.solutions.holistic  # Holistic model
mp_drawing = mp.solutions.drawing_utils  # Drawing utilities


def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False  # Image is no longer writeable
    results = model.process(image)  # Make prediction
    image.flags.writeable = True  # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR COVERSION RGB 2 BGR
    return image, results


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)  # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw right hand connections


def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                              )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                              )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                              )


def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in
                     results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    lh = np.array([[res.x, res.y, res.z] for res in
                   results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in
                   results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(
        21 * 3)
    return np.concatenate([pose, lh, rh])

In [18]:
class Video():

    def __init__(self):
        self.video_dataset = self.generate_dataset()
        self.number_of_videos = 30
        self.number_of_frames = 30
        self.path = os.path.join('EXTRACTION')
        
    def update_dataset_collection(self, new_dataset):

        self.video_dataset = self.video_dataset | new_dataset
        
    def get_video_dataset(self):
        return self.video_dataset

    def get_videos(self, video_query):
        for video_category in self.video_dataset:
            if video_query == video_category:
                return self.video_dataset[video_query]

    def get_video_sequence_length(self, video):
        cap = cv2.VideoCapture(video)
        total_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        return total_frame_count

    def generate_dataset(self):
        directory = "DATASET"

        # create a dictionary
        # to store video directory and the amount of frames of each video
        videos_array = dict()

        for filename in os.listdir(directory):

            file = os.path.join(directory, filename)

            # Store the name of each dance at the start of the dictionary
            videos_array[filename] = []

            for video_filename in os.listdir(file):
                video_dict = dict()

                # Gets specified video file directory
                video_file = os.path.join(file, video_filename)

                video_dict['File'] = video_file
                
                # Get frame length of each video
                video_dict['Frames'] = self.get_video_sequence_length(video_file)

                # Store frame length to each respect individual video
                videos_array[filename].append(video_dict)

                # Create an action array to store numpy data
        actions = np.array(list(videos_array))
        return videos_array

    
    def directory_to_store_video_keypoints(self):

        for dance in list(self.video_dataset):
            number_of_videos = len(self.video_dataset[dance])
            for dictionary in self.video_dataset[dance]:
                for video_number in range(self.number_of_videos):
                    try:
                        os.makedirs(os.path.join(self.path, dance, str(video_number)))
                    except:
                        pass
    
    def collect_video_keypoints(self):
        """This will allow a user to capture new movements from a video for the program to then
        collect the new keypoints as npy file to then train the model"""
        
        self.directory_to_store_video_keypoints()

        for dance in list(self.video_dataset):
            current_video = -1

            # Loop through all videos of dance
            for dictionary in self.video_dataset[dance]:

                video = dictionary['File']
                
                current_video += 1
                print("this is video file: " + video)

                cap = cv2.VideoCapture(video)

                # Set mediapipe model
                with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

                    print("this is video number: " + str(current_video))

                    # Loop through video length aka sequence length
                    for frame_num in range(self.number_of_frames):

                        # Read feed
                        ret, frame = cap.read()
                        if not ret:

                            print("Ignoring empty camera frame.")
                            break
                        else:
                            # Make detections
                            image, results = mediapipe_detection(frame, holistic)

                            # Draw landmarks
                            draw_styled_landmarks(image, results)

                            # NEW Apply wait logic
                            if frame_num == 0:
                                cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(dance, current_video),
                                            (15, 12),
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                                # Show to screen
                                cv2.imshow('OpenCV Feed', image)
                                cv2.waitKey(2000)
                            else:
                                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(dance, current_video),
                                            (15, 12),
                                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                                # Show to screen
                                cv2.imshow('OpenCV Feed', image)

                            # NEW Export keypoints
                            keypoints = extract_keypoints(results)
                            npy_path = os.path.join(self.path, dance, str(current_video), str(frame_num))
                            np.save(npy_path, keypoints)

                            # Break gracefully
                            if cv2.waitKey(10) & 0xFF == ord('q'):
                                break

                    cap.release()
                    cv2.destroyAllWindows()       

In [19]:
class RecordVideo(Video):

    def __init__(self, new_action_list):
        """new_action_list must be an array"""
        super().__init__()
        # Thirty videos worth of data
        self.no_sequences = 1
        # Videos are going to be 30 frames in length
        self.sequence_length = 30
        # Directory where videos are being stored
        self.PATH = os.path.join('DATASET')
        # initialize Text-to-speech engine
        self.engine = pyttsx3.init()

        self.new_actions = new_action_list

    def text_to_speech(self, text):
        """Convert text into speech"""
        self.engine.say(text)

        # play the speech
        self.engine.runAndWait()

    def make_video_src_directory(self, actions_list):
        """Create a directory for the new action being captured"""
        for action in actions_list:
            try:
                os.makedirs(os.path.join(self.PATH, action))
            except:
                pass

    def capture_new_dance_action(self):
        """Record the new action 30 times and store it in the respective folder"""  
        self.make_video_src_directory(self.new_actions)
        temp_string = ""
        action_dictionary = dict()
        video_set_array = []
        
        for dance_action in self.new_actions:
            
            temp_string = dance_action
                        
            n = 0
            print("Capturing Dance Videos For: " + dance_action)
            self.text_to_speech("Capturing Dance Videos For: " + dance_action)
            for i in range(self.no_sequences):

                # The duration in seconds of the video captured

                print("Capturing " + dance_action + " Video Number: "  + str(n+1))
                self.text_to_speech("Video Number: " + str(n+1))
                
                individual_video_dictionary = dict()
                
                
                capture_duration = 4

                cap = cv2.VideoCapture(1)

                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

                if n <= 9:
                    name = dance_action + "_" + "00" + str(n) + ".mp4"

                elif 99 >= n > 9:
                    name = dance_action + "_" + "0" + str(n) + ".mp4"
                else:
                    name = dance_action + "_" + str(n) + ".mp4"

                
                mp4_path = os.path.join(self.PATH, dance_action, name)
                fourcc = cv2.VideoWriter_fourcc(*'MP4V')
                print(mp4_path)
                out = cv2.VideoWriter(mp4_path, fourcc, 30.0, (width, height), isColor=True)
                n += 1
                start_time = time.time()
                self.text_to_speech("Starting Recording")
                while int(time.time() - start_time) < capture_duration:

                    ret, frame = cap.read()
                    if ret:

                        out.write(frame)
                        cv2.putText(frame, 'COLLECTING DATASET', (120, 200),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                        cv2.putText(frame, 'Collecting frames for {} Video Number {}'.format(dance_action, n),
                                    (15, 12),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        cv2.imshow('frame', frame)

                        key = cv2.waitKey(1)
                        if key == ord('q'):
                            break
                        if key == ord('p'):
                            cv2.waitKey(-1)  # wait until any key is pressed

                    else:
                        break
                
                
                individual_video_dictionary['File'] = mp4_path
                individual_video_dictionary['Frames'] = 0
                
                video_set_array.append(individual_video_dictionary)
                
                
                self.text_to_speech("Stop Recording")
    
                
                cap.release()
                out.release()
                cv2.destroyAllWindows()
        action_dictionary[temp_string] = video_set_array
        self.text_to_speech("All Recordings Complete. Have a great day!")
        
        return action_dictionary


In [20]:
class Action(Video):

    def __init__(self, video):
        super().__init__()
        
        # pass in object video
        self.video = video
        
        # Store action collection
        self.actions = np.array(sorted(video.get_video_dataset()))
    
    def get_video(self):
        return self.video
    
    def get_actions(self):
        return self.actions
    
    def search_action(self, query):
        for action in self.actions:
            if query == action:
                return action
        return None

    def add_new_action(self):
        
        print("We will need to capture this action 30 times\n")
        print("Enter q to quit\n")
        print("Make sure the name of action is at least  3 characters\n")
        x = True
        while x:

            new_action = str(input('Enter the name of your dance action, the action has to be short to be captured: '))

            if new_action == "q" or new_action == "Q":
                print("Process terminated")
                return None

            elif len(new_action) < 3:
                print("The action name must be longer at least 3 characters.\n") 

            elif new_action in self.actions:
                print("Unfortunately an action of this name already exists.\n")
            else:
                x = False

        print("Action about to be captured")
        
        self.actions= np.append(self.actions, new_action)
        
        new_action_dataset = RecordVideo([new_action]).capture_new_dance_action()
        
        self.video.update_dataset_collection(new_action_dataset)
        

        


In [44]:
class KerasModel(Action):
    
    
    def __init__(self, action):
        
        # this is the number of frames we are using from a video
        self.sequence_length = 30
        self.path = os.path.join('EXTRACTION')
        
        # action is an object
        # actions is a list
        self.actions = action.actions
        
        self.log_dir = os.path.join('Logs')
        self.tb_callback = TensorBoard(log_dir=self.log_dir)
        self.model = Sequential()
        self.model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 258)))
        self.model.add(LSTM(128, return_sequences=True, activation='relu'))
        self.model.add(LSTM(64, return_sequences=False, activation='relu'))
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(32, activation='relu'))
        self.model.add(Dense(self.actions.shape[0], activation='softmax'))
        self.model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

    def get_model(self):
        return self.model
        
        
    def set_model(self, load):
        X_train, X_test, y_train, y_test = self.train()

        if load == False:
            self.model.fit(X_train, y_train, epochs=80, callbacks=[self.tb_callback])
            self.model.save('DTHOT_3.h5')

        else:
            self.model.fit(X_train, y_train, epochs=0, callbacks=[self.tb_callback])
            self.model.load_weights('DTHOT.h5')
            print("Model loaded succesfully")
        
        
    def train(self):
        
        label_map = {label: num for num, label in enumerate(self.actions)}
         
        sequences, labels = [], []
        for action in self.actions:
            for sequence in np.array(os.listdir(os.path.join(self.path, action))).astype(int):
                window = []
                for frame_num in range(self.sequence_length):
                    res = np.load(os.path.join(self.path, action, str(sequence), "{}.npy".format(frame_num)))
                    window.append(res)
                sequences.append(window)
                labels.append(label_map[action])

        X = np.array(sequences)
        y = to_categorical(labels).astype(int)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
        return X_train, X_test, y_train, y_test

    
    def model_statistics(self):
        print(self.model.summary())

    



In [22]:
class Routine(Action):

    def __init__(self, action, boolean):
        # pass in the object action
        self.action = action
        if boolean == True:
            with open('routine_catalogue.pkl', 'rb') as file:
                self.routines = pickle.load(file)
        else:
            self.routines = {}

    def update_routines(self, new_routine):
        self.routines = self.routines | new_routine

    def get_routines(self):
        return self.routines

    def store_routine(self):
        # Open a file and use dump()
        with open('routine_catalogue.pkl', 'wb') as file:
            # A new file will be created
            pickle.dump(self.routines, file)

    def add_routine(self):

        boolean_3 = True

        while boolean_3:
            print("Enter q to quit process\n")
            new_routine = str(input('Enter the name of your dance routine: \n')).lower()

            if new_routine == "q":
                print("Terminating Process")
                return None

            elif new_routine in sorted(self.routines):
                print("A routine of this name exists")
            else:
                boolean_3 = False

        action_list_of_routine = []
        routine_to_add = dict()

        print("Actions list: \n")

        for action_item in self.action.actions:
            option_number = np.where(self.action.actions == action_item)[0]

            print(str(option_number) + ": " + action_item)

        boolean_2 = True

        while boolean_2:
            print("Enter -1 to quit process\n")
            number_of_actions = int(input("Enter how many actions you wish to add to your routine: \n"))

            if number_of_actions == -1:
                print("Terminating Process")
                return None

            elif number_of_actions <= 2:
                print("You have to enter at minimum 3 actions to make a routine\n ")
            else:
                boolean_2 = False

        counter = 0
        boolean = True

        while boolean:
            print("\nEnter -1 to quit process\n")
            action_index = int(input("Enter action associted to the number to add to routine: \n"))

            if action_index == -1:
                print("Terminating Process")
                return None

            elif action_index >= len(self.action.actions) or action_index < -1:
                print("There's no action associated with this index\n")

            else:
                action_list_of_routine.append(self.action.actions[action_index])
                counter += 1
                print("Action: " + self.action.actions[action_index] + " added successfully!")
                if number_of_actions == counter:
                    boolean = False

        print("Adding your routine to catalogue....\n")
        print("\nHere is your dance routine: ", action_list_of_routine)

        routine_to_add[new_routine] = action_list_of_routine

        self.update_routines(routine_to_add)
        self.store_routine()

    def find_routine(self, captured_actions):
        """Takes an array of actions from user. Example format:['hop', 'jump', 'kick', 'kick']  """
        possible_routine_list = []

        for routine in self.routines:
            score = fuzz.ratio(captured_actions, self.routines[routine])
            highest_score = score
            most_similar_routine = routine

            if score == 100:
                print("We found the routine you just did: ", routine)
                self.display_score(score)
                return None
            elif score >= 55:
                possible_routine_list.append(routine)
            elif score > highest_score and len(possible_routine_list) == 0:
                highest_score = score
                most_similar_routine = routine
        
        if highest_score < 20:
            print("We could not detect a routine with these dance steps ")        
            
        elif len(possible_routine_list) == 0:
            print("This is the most similar dance routine we could find: ", most_similar_routine)
            self.get_incorrect_steps([most_similar_routine], captured_actions)
            self.display_score(score)
            
        else:
            print("Here are the routine(s) with similar dance steps : ", possible_routine_list)
            self.get_incorrect_steps(possible_routine_list, captured_actions)
            self.display_score(score)
            
    def get_incorrect_steps(self, possible_routines, user_routine):
        
        for routine in possible_routines:
            
            print("For "+ routine + " here are the steps you need to work on\n")
            
            routine_actions = self.routines[routine]
            index = 0
            
            if len(routine_actions) < len(user_routine):
                for i in range(len(routine_actions)):
                    if routine_actions[i] != user_routine[i]:
                        print("At dance step "+ str(i+1) + " of dance routine " + routine +".\n")
                        print("You did "+ user_routine[i]+ " the correct move should had been " + routine_actions[i]+"\n")
                        index = i
                print("Here are some actions in your performace we found are not in the routine: ")       
                for i in range(index, len(user_routine)):
                    print(user_routine[i])
                print("\n")
                    
            elif len(user_routine) < len(routine_actions):
                for i in range(len(user_routine)):
                    if routine_actions[i] != user_routine[i]:
                        print("At dance step "+ str(i+1) + " of dance routine " + routine +".\n")
                        print("You did "+ user_routine[i]+ " the correct move should had been " + routine_actions[i]+"\n")
                        index = i
                print("There are a few actions in your performace we found you are missing from your routine: ")       
                for i in range(index, len(routine_actions)):
                    print(routine_actions[i])
                print("\n")
                
            else:
                for i in range(len(user_routine)):
                    if routine_actions[i] != user_routine[i]:
                        print("At dance step "+ str(i+1) + " of dance routine " + routine +".\n")
                        print("You did "+ user_routine[i]+ " the correct move should had been " + routine_actions[i]+"\n")
                
    def display_score(self, score):
        
        if score == 100:
            print("You scored a perfect 100! You are a dancing prodigy!")   
        elif 70 <= score <= 99:
            print("Brilliant! You are close to perfection. You scored: "+ str(score)+"\n")
        elif 40 <= score <= 69:
            print("You need to practice more. You scored " + str(score)+"\n")
        elif score <= 39:
             print("At least you have personality. You scored " + str(score)+"\n")
        
        
        


In [23]:

colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245), (19, 17, 245), (190, 117, 245), (190, 117, 245),
          (190, 117, 245), (190, 117, 245), (190, 117, 245), (190, 117, 245)]


def prob_viz(res, actions, input_frame):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                    cv2.LINE_AA)

    return output_frame



def go_live(model,actions,routine):
    sequence = []
    dance_moves = []
    threshold = 0.5

    cap = cv2.VideoCapture(0)
    # Set mediapipe model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():

            # Read feed
            ret, frame = cap.read()
            # Make detections
            image, results = mediapipe_detection(frame, holistic)

            # Draw landmarks
            draw_styled_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
            sequence.append(keypoints)
            sequence = sequence[-30:]

            if results.pose_landmarks is None:
                cv2.putText(image, 'Please have your whole', (60, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, cv2.LINE_AA)
                cv2.putText(image, 'body in frame', (60, 250),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, cv2.LINE_AA)

            else:

#                 for data_point in results.pose_landmarks.landmark:
#                     if data_point.visibility < 0.5:
#                         cv2.putText(image, "Having a hard time detecting you", (60, 300),
#                                     cv2.FONT_HERSHEY_SIMPLEX, 1, (35, 200, 55), 4, cv2.LINE_AA)
#                         cv2.putText(image, 'Area needs to be well lit', (60, 350),
#                                     cv2.FONT_HERSHEY_SIMPLEX, 1, (35, 200, 55), 4, cv2.LINE_AA)

                if len(sequence) == 30:
                    res = model.predict(np.expand_dims(sequence, axis=0))[0]

                    if actions[np.argmax(res)] == "stop_capturing_movement" :
                        routine.find_routine(dance_moves)                        
                        cv2.putText(image, "Capturing has stopped", (60, 300),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (35, 200, 55), 4, cv2.LINE_AA)
                    else:

                        # 3. Viz logic
                        if res[np.argmax(res)] > threshold:
                            if len(dance_moves) > 0:
                                if actions[np.argmax(res)] != dance_moves[-1]:
                                    dance_moves.append(actions[np.argmax(res)])
                            else:
                                dance_moves.append(actions[np.argmax(res)])
                                routine.find_routine(dance_moves)

                        if len(dance_moves) > 5:
                            dance_moves = dance_moves[-5:]

                        # Viz probabilities
#                         image = prob_viz(res, actions, image)

                    cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
                    cv2.putText(image,
                                ' '.join(dance_moves),
                                (3, 30),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.6,
                                (255, 255, 255),
                                2,
                                cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

In [24]:
dataset = Video()

In [30]:
dataset.collect_video_keypoints()

this is video file: DATASET\kick_right_leg\kick_right_leg_000.mp4
this is video number: 0
this is video file: DATASET\kick_right_leg\kick_right_leg_001.mp4
this is video number: 1
this is video file: DATASET\kick_right_leg\kick_right_leg_002.mp4
this is video number: 2
this is video file: DATASET\kick_right_leg\kick_right_leg_003.mp4
this is video number: 3
this is video file: DATASET\kick_right_leg\kick_right_leg_004.mp4
this is video number: 4
this is video file: DATASET\kick_right_leg\kick_right_leg_005.mp4
this is video number: 5
this is video file: DATASET\kick_right_leg\kick_right_leg_006.mp4
this is video number: 6
this is video file: DATASET\kick_right_leg\kick_right_leg_007.mp4
this is video number: 7
this is video file: DATASET\kick_right_leg\kick_right_leg_008.mp4
this is video number: 8
this is video file: DATASET\kick_right_leg\kick_right_leg_009.mp4
this is video number: 9
this is video file: DATASET\kick_right_leg\kick_right_leg_010.mp4
this is video number: 10
this is v

this is video file: DATASET\start_capturing_movement\start_capturing_movement_015.mp4
this is video number: 15
this is video file: DATASET\start_capturing_movement\start_capturing_movement_016.mp4
this is video number: 16
this is video file: DATASET\start_capturing_movement\start_capturing_movement_017.mp4
this is video number: 17
this is video file: DATASET\start_capturing_movement\start_capturing_movement_018.mp4
this is video number: 18
this is video file: DATASET\start_capturing_movement\start_capturing_movement_019.mp4
this is video number: 19
this is video file: DATASET\start_capturing_movement\start_capturing_movement_020.mp4
this is video number: 20
this is video file: DATASET\start_capturing_movement\start_capturing_movement_021.mp4
this is video number: 21
this is video file: DATASET\start_capturing_movement\start_capturing_movement_022.mp4
this is video number: 22
this is video file: DATASET\start_capturing_movement\start_capturing_movement_023.mp4
this is video number: 23
t

this is video file: DATASET\turn_180_around_fist_up_sway_hips\turn_180_around_fist_up_sway_hips_003.mp4
this is video number: 3
this is video file: DATASET\turn_180_around_fist_up_sway_hips\turn_180_around_fist_up_sway_hips_004.mp4
this is video number: 4
this is video file: DATASET\turn_180_around_fist_up_sway_hips\turn_180_around_fist_up_sway_hips_005.mp4
this is video number: 5
this is video file: DATASET\turn_180_around_fist_up_sway_hips\turn_180_around_fist_up_sway_hips_006.mp4
this is video number: 6
this is video file: DATASET\turn_180_around_fist_up_sway_hips\turn_180_around_fist_up_sway_hips_007.mp4
this is video number: 7
this is video file: DATASET\turn_180_around_fist_up_sway_hips\turn_180_around_fist_up_sway_hips_008.mp4
this is video number: 8
this is video file: DATASET\turn_180_around_fist_up_sway_hips\turn_180_around_fist_up_sway_hips_009.mp4
this is video number: 9
this is video file: DATASET\turn_180_around_fist_up_sway_hips\turn_180_around_fist_up_sway_hips_010.mp4


In [31]:
action_list = Action(Video())

In [32]:
action_list.actions

array(['kick_right_leg', 'point_left_hand_around_bounce_hips',
       'start_capturing_movement', 'stop_capturing_movement',
       'sweep_hands_right', 'turn_180_around_fist_up_sway_hips'],
      dtype='<U34')

In [20]:
action_list.add_new_action()

We will need to capture this action 30 times

Enter q to quit

Make sure the name of action is at least  3 characters

Enter the name of your dance action, the action has to be short to be captured: q
Process terminated


In [45]:
dthot = KerasModel(action_list)

In [46]:
dthot.set_model(False)

Epoch 1/80
6/6 [==============================] - 2s 22ms/step - loss: 1.7657 - categorical_accuracy: 0.1345
Epoch 2/80
6/6 [==============================] - 0s 20ms/step - loss: 1.4329 - categorical_accuracy: 0.3918
Epoch 3/80
6/6 [==============================] - 0s 20ms/step - loss: 0.6585 - categorical_accuracy: 0.7193
Epoch 4/80
6/6 [==============================] - 0s 21ms/step - loss: 1.4775 - categorical_accuracy: 0.6374
Epoch 5/80
6/6 [==============================] - 0s 20ms/step - loss: 0.6378 - categorical_accuracy: 0.7953
Epoch 6/80
6/6 [==============================] - 0s 22ms/step - loss: 0.5314 - categorical_accuracy: 0.8830
Epoch 7/80
6/6 [==============================] - 0s 20ms/step - loss: 0.4019 - categorical_accuracy: 0.9006
Epoch 8/80
6/6 [==============================] - 0s 21ms/step - loss: 0.2572 - categorical_accuracy: 0.9181
Epoch 9/80
6/6 [==============================] - 0s 21ms/step - loss: 0.1508 - categorical_accuracy: 0.9591
Epoch 10/80
6/6 [==

6/6 [==============================] - 0s 21ms/step - loss: 0.0546 - categorical_accuracy: 0.9766
Epoch 76/80
6/6 [==============================] - 0s 22ms/step - loss: 0.0835 - categorical_accuracy: 0.9766
Epoch 77/80
6/6 [==============================] - 0s 22ms/step - loss: 0.0994 - categorical_accuracy: 0.9591
Epoch 78/80
6/6 [==============================] - 0s 20ms/step - loss: 0.0583 - categorical_accuracy: 0.9825
Epoch 79/80
6/6 [==============================] - 0s 20ms/step - loss: 0.0556 - categorical_accuracy: 0.9883
Epoch 80/80
6/6 [==============================] - 0s 21ms/step - loss: 0.1429 - categorical_accuracy: 0.9415


In [47]:
routine = Routine(action_list, True)

In [48]:
routine.routines

{'test': ['sweep_hands_right',
  'sweep_hands_right',
  'stop_capturing_movement',
  'stop_capturing_movement',
  'start_capturing_movement'],
 'test 2': ['kick_right_leg',
  'turn_180_around_fist_up_sway_hips',
  'point_left_hand_around_bounce_hips']}

In [15]:
routine.find_routine(['kick_right_leg', 'point_left_hand_around_bounce_hips', 'sweep_hands_right', 'kick', 'spit', 'lick'])

Here are the routine(s) with similar dance steps :  ['test 2']
For test 2 here are the steps you need to work on

At dance step 2 of dance routine test 2.

You did point_left_hand_around_bounce_hips the correct move should had been turn_180_around_fist_up_sway_hips

At dance step 3 of dance routine test 2.

You did sweep_hands_right the correct move should had been point_left_hand_around_bounce_hips

Here are some actions in your performace we found are not in the routine: 
sweep_hands_right
kick
spit
lick


You need to practice more. You scored 58



In [181]:
routine.find_routine(['kick_right_leg', 'point_left_hand_around_bounce_hips'])

Here are the routine(s) with similar dance steps :  ['test 2']
For test 2 here are the steps you need to work on

At dance step 2 of dance routine test 2.

You did point_left_hand_around_bounce_hips the correct move should had been turn_180_around_fist_up_sway_hips

There are a few actions in your performace we found you are missing from your routine: 
turn_180_around_fist_up_sway_hips
point_left_hand_around_bounce_hips


Brilliant! You are close to perfection. You scored: 75



In [185]:
routine.find_routine(['kick_right_leg', 'turn_180_around_fist_up_sway_hips', 'point_left_hand_around_bounce_hips'])

We found the routine you just did:  test 2
You scored a perfect 100! You are a dancing prodigy!


In [186]:
['kick_right_leg', 'turn_180_around_fist_up_sway_hips', 'point_left_hand_around_bounce_hips']

['kick_right_leg',
 'turn_180_around_fist_up_sway_hips',
 'point_left_hand_around_bounce_hips']

In [187]:
routine.find_routine(['hop', 'turn_180_around_fist_up_sway_hips', 'point_left_hand_around_bounce_hips'])

Here are the routine(s) with similar dance steps :  ['test 2']
For test 2 here are the steps you need to work on

At dance step 1 of dance routine test 2.

You did hop the correct move should had been kick_right_leg

Brilliant! You are close to perfection. You scored: 91



In [49]:
go_live(dthot.model, action_list.actions, routine)

This is the most similar dance routine we could find:  test 2
For test 2 here are the steps you need to work on

At dance step 1 of dance routine test 2.

You did start_capturing_movement the correct move should had been kick_right_leg

There are a few actions in your performace we found you are missing from your routine: 
kick_right_leg
turn_180_around_fist_up_sway_hips
point_left_hand_around_bounce_hips


At least you have personality. You scored 25



In [ ]:
action_list.add_new_action()

In [27]:
routine.add_routine()

Enter q to quit process

Enter the name of your dance routine: 
test 2
Actions list: 

[0]: kick_right_leg
[1]: point_left_hand_around_bounce_hips
[2]: start_capturing_movement
[3]: stop_capturing_movement
[4]: sweep_hands_right
[5]: turn_180_around_fist_up_sway_hips
Enter -1 to quit process

Enter how many actions you wish to add to your routine: 
0
You have to enter at minimum 3 actions to make a routine
 
Enter -1 to quit process

Enter how many actions you wish to add to your routine: 
3

Enter -1 to quit process

Enter action associted to the number to add to routine: 
0
Action: kick_right_leg added successfully!

Enter -1 to quit process

Enter action associted to the number to add to routine: 
5
Action: turn_180_around_fist_up_sway_hips added successfully!

Enter -1 to quit process

Enter action associted to the number to add to routine: 
1
Action: point_left_hand_around_bounce_hips added successfully!
Adding your routine to catalogue....


Here is your dance routine:  ['kick_ri